# Neural Collaborative Filtering
Building a **Neural Matrix Factorization** model from scratch in PyTorch on MovieLens-100k dataset. 

In [1]:
!pip install -q tensorboardX

In [2]:
import os
import time
import random
import argparse
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tensorboardX import SummaryWriter

## Downloading Movielens-100k Ratings

In [3]:
DATA_URL = "https://raw.githubusercontent.com/SudeshGowda/ml-100k-dataset/main/u.data"
MAIN_PATH = '/content/'
DATA_PATH = MAIN_PATH + 'u.data'
MODEL_PATH = MAIN_PATH + 'models/'
MODEL_NEUMF = 'ml-100k_NeuMF'
MODEL_GMF = 'ml-100k_GMF'
MODEL_MLP = 'ml-100k_MLP'

In [4]:
!wget -nc https://raw.githubusercontent.com/SudeshGowda/ml-100k-dataset/main/u.data

File ‘u.data’ already there; not retrieving.



In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

## Defining Dataset Classes

In [6]:
class Rating_Datset(torch.utils.data.Dataset):
	def __init__(self, user_list, item_list, rating_list):
		super(Rating_Datset, self).__init__()
		self.user_list = user_list
		self.item_list = item_list
		self.rating_list = rating_list

	def __len__(self):
		return len(self.user_list)

	def __getitem__(self, idx):
		user = self.user_list[idx]
		item = self.item_list[idx]
		rating = self.rating_list[idx]
		
		return (
			torch.tensor(user, dtype=torch.long),
			torch.tensor(item, dtype=torch.long),
			torch.tensor(rating, dtype=torch.float)
			)

### NCF Dataset Class
- *_reindex*: process dataset to reindex userID and itemID, also set rating as binary feedback
- *_leave_one_out*: leave-one-out evaluation
- *negative_sampling*: randomly selects n negative examples for each positive one

In [7]:
class NCF_Data(object):
	"""
	Construct Dataset for NCF
	"""
	def __init__(self, args, ratings):
		self.ratings = ratings
		self.num_ng = args.num_ng
		self.num_ng_test = args.num_ng_test
		self.batch_size = args.batch_size

		self.preprocess_ratings = self._reindex(self.ratings)

		self.user_pool = set(self.ratings['user_id'].unique())
		self.item_pool = set(self.ratings['item_id'].unique())

		self.train_ratings, self.test_ratings = self._leave_one_out(self.preprocess_ratings)
		self.negatives = self._negative_sampling(self.preprocess_ratings)
		random.seed(args.seed)
	
	def _reindex(self, ratings):
		"""
		Process dataset to reindex userID and itemID, also set rating as binary feedback
		"""
		user_list = list(ratings['user_id'].drop_duplicates())
		user2id = {w: i for i, w in enumerate(user_list)}

		item_list = list(ratings['item_id'].drop_duplicates())
		item2id = {w: i for i, w in enumerate(item_list)}

		ratings['user_id'] = ratings['user_id'].apply(lambda x: user2id[x])
		ratings['item_id'] = ratings['item_id'].apply(lambda x: item2id[x])
		ratings['rating'] = ratings['rating'].apply(lambda x: float(x > 0))
		return ratings

	def _leave_one_out(self, ratings):
		ratings['rank_latest'] = ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)
		test = ratings.loc[ratings['rank_latest'] == 1]
		train = ratings.loc[ratings['rank_latest'] > 1]
		assert train['user_id'].nunique()==test['user_id'].nunique(), 'Not Match Train User with Test User'
		return train[['user_id', 'item_id', 'rating']], test[['user_id', 'item_id', 'rating']]

	def _negative_sampling(self, ratings):
		interact_status = (
			ratings.groupby('user_id')['item_id']
			.apply(set)
			.reset_index()
			.rename(columns={'item_id': 'interacted_items'}))
		interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: self.item_pool - x)
		interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, self.num_ng_test))
		return interact_status[['user_id', 'negative_items', 'negative_samples']]

	def get_train_instance(self):
		users, items, ratings = [], [], []
		train_ratings = pd.merge(self.train_ratings, self.negatives[['user_id', 'negative_items']], on='user_id')
		train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, self.num_ng))
		for row in train_ratings.itertuples():
			users.append(int(row.user_id))
			items.append(int(row.item_id))
			ratings.append(float(row.rating))
			for i in range(self.num_ng):
				users.append(int(row.user_id))
				items.append(int(row.negatives[i]))
				ratings.append(float(0))  # negative samples get 0 rating
		dataset = Rating_Datset(
			user_list=users,
			item_list=items,
			rating_list=ratings)
		return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=4)

	def get_test_instance(self):
		users, items, ratings = [], [], []
		test_ratings = pd.merge(self.test_ratings, self.negatives[['user_id', 'negative_samples']], on='user_id')
		for row in test_ratings.itertuples():
			users.append(int(row.user_id))
			items.append(int(row.item_id))
			ratings.append(float(row.rating))
			for i in getattr(row, 'negative_samples'):
				users.append(int(row.user_id))
				items.append(int(i))
				ratings.append(float(0))
		dataset = Rating_Datset(
			user_list=users,
			item_list=items,
			rating_list=ratings)
		return torch.utils.data.DataLoader(dataset, batch_size=self.num_ng_test+1, shuffle=False, num_workers=2)

## Defining Metrics
Using Hit Rate and NDCG as our evaluation *metrics*

In [8]:
def hit(ng_item, pred_items):
	if ng_item in pred_items:
		return 1
	return 0


def ndcg(ng_item, pred_items):
	if ng_item in pred_items:
		index = pred_items.index(ng_item)
		return np.log2(2)/np.log2(index+2)
	return 0


def metrics(model, test_loader, top_k, device):
	HR, NDCG = [], []

	for user, item, label in test_loader:
		user = user.to(device)
		item = item.to(device)

		predictions = model(user, item)
		_, indices = torch.topk(predictions, top_k)
		recommends = torch.take(
				item, indices).cpu().numpy().tolist()

		ng_item = item[0].item() # leave one-out evaluation has only one item per user
		HR.append(hit(ng_item, recommends))
		NDCG.append(ndcg(ng_item, recommends))

	return np.mean(HR), np.mean(NDCG)

### Defining Model Architectures
1. Generalized Matrix Factorization
2. Multi Layer Perceptron
3. Neural Matrix Factorization

In [9]:
class Generalized_Matrix_Factorization(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(Generalized_Matrix_Factorization, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args.factor_num

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.affine_output = nn.Linear(in_features=self.factor_num, out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating.squeeze()

    def init_weight(self):
        pass

In [10]:
class Multi_Layer_Perceptron(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(Multi_Layer_Perceptron, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args.factor_num
        self.layers = args.layers

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = nn.ReLU()(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

    def init_weight(self):
        pass

In [11]:
class NeuMF(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(NeuMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num_mf = args.factor_num
        self.factor_num_mlp =  int(args.layers[0]/2)
        self.layers = args.layers

        self.embedding_user_mlp = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mlp)

        self.embedding_user_mf = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mf)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(args.layers[:-1], args.layers[1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
            self.fc_layers.append(nn.ReLU())

        self.affine_output = nn.Linear(in_features=args.layers[-1] + self.factor_num_mf, out_features=1)
        self.logistic = nn.Sigmoid()
        self.init_weight()

    def init_weight(self):
        nn.init.normal_(self.embedding_user_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_user_mf.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mf.weight, std=0.01)
        
        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                
        nn.init.xavier_uniform_(self.affine_output.weight)

        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, user_indices, item_indices):
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)

        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)

        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)
        mf_vector =torch.mul(user_embedding_mf, item_embedding_mf)

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

### Setting Arguments

Here is the brief description of important ones:
- Learning rate is 0.001
- HitRate@10 and NDCG@10
- 4 negative samples for each positive one

In [12]:
#collapse-hide
parser = argparse.ArgumentParser()
parser.add_argument("--seed", 
	type=int, 
	default=42, 
	help="Seed")
parser.add_argument("--lr", 
	type=float, 
	default=0.001, 
	help="learning rate")
parser.add_argument("--batch_size", 
	type=int, 
	default=256, 
	help="batch size for training")
parser.add_argument("--neumf_epochs", 
	type=int,
	default=10,  
	help="training epoches for neumf")
parser.add_argument("--mlp_epochs", 
	type=int,
	default=20,  
	help="training epoches for neumf")
parser.add_argument("--gmf_epochs", 
	type=int,
	default=20,  
	help="training epoches for neumf")
parser.add_argument("--top_k", 
	type=int, 
	default=10, 
	help="compute metrics@top_k")
parser.add_argument("--factor_num", 
	type=int,
	default=32, 
	help="predictive factors numbers in the model")
parser.add_argument("--layers",
    nargs='+', 
    default=[64,32,16,8],
    help="MLP layers. Note that the first layer is the concatenation of user \
    and item embeddings. So layers[0]/2 is the embedding size.")
parser.add_argument("--num_ng", 
	type=int,
	default=4, 
	help="Number of negative samples for training set")
parser.add_argument("--num_ng_test", 
	type=int,
	default=100, 
	help="Number of negative samples for test set")
parser.add_argument("--out", 
	default=True,
	help="save model or not")

_StoreAction(option_strings=['--out'], dest='out', nargs=None, const=None, default=True, type=None, choices=None, help='save model or not', metavar=None)

In [13]:
# set device and parameters
args = parser.parse_args("")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# seed for Reproducibility
seed_everything(args.seed)

# load data
ml_100k = pd.read_csv(
	DATA_PATH, 
	sep="\t", 
	names = ['user_id', 'item_id', 'rating', 'timestamp'], 
	engine='python')

# set the num_users, items
num_users = ml_100k['user_id'].nunique()+1
num_items = ml_100k['item_id'].nunique()+1

# construct the train and test datasets
data = NCF_Data(args, ml_100k)
train_loader = data.get_train_instance()
test_loader = data.get_test_instance()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Training NeuMF Model

In [14]:
writer = SummaryWriter()
# set model and loss, optimizer
model = NeuMF(args, num_users, num_items)
model = model.to(device)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation
best_hr = 0
for epoch in range(1, args.neumf_epochs+1):
    model.train()
    start_time = time.time()

    for user, item, label in train_loader:
        user = user.to(device)
        item = item.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        prediction = model(user, item)
        loss = loss_function(prediction, label)
        loss.backward()
        optimizer.step()
        writer.add_scalar('NeuMF/loss/Train_loss', loss.item(), epoch)

    model.eval()
    HR, NDCG = metrics(model, test_loader, args.top_k, device)
    writer.add_scalar('NeuMF/Perfomance/HR@10', HR, epoch)
    writer.add_scalar('NeuMF/Perfomance/NDCG@10', NDCG, epoch)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
            time.str!lsftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    if HR > best_hr:
        best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
        if args.out:
            if not os.path.exists(MODEL_PATH):
                os.mkdir(MODEL_PATH)
            torch.save(model, 
                '{}{}.pth'.format(MODEL_PATH, MODEL_NEUMF))

writer.close()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


The time elapse of epoch 001 is: 00: 00: 26
HR: 0.549	NDCG: 0.309
The time elapse of epoch 002 is: 00: 00: 26
HR: 0.602	NDCG: 0.343
The time elapse of epoch 003 is: 00: 00: 26
HR: 0.630	NDCG: 0.361
The time elapse of epoch 004 is: 00: 00: 26
HR: 0.645	NDCG: 0.373
The time elapse of epoch 005 is: 00: 00: 26
HR: 0.657	NDCG: 0.385
The time elapse of epoch 006 is: 00: 00: 26
HR: 0.657	NDCG: 0.386
The time elapse of epoch 007 is: 00: 00: 27
HR: 0.653	NDCG: 0.378
The time elapse of epoch 008 is: 00: 00: 26
HR: 0.644	NDCG: 0.373
The time elapse of epoch 009 is: 00: 00: 27
HR: 0.633	NDCG: 0.365
The time elapse of epoch 010 is: 00: 00: 27
HR: 0.617	NDCG: 0.359


In [15]:
print("NeuMF best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(best_epoch, best_hr, best_ndcg))

NeuMF best epoch 005: HR = 0.657, NDCG = 0.385


## Training MLP model


In [16]:
writer = SummaryWriter()
# set model and loss, optimizer
model = Multi_Layer_Perceptron(args, num_users, num_items)
model = model.to(device)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation

best_hr = 0
for epoch in range(1, args.mlp_epochs+1):
	model.train()
	start_time = time.time()

	for user, item, label in train_loader:
		user = user.to(device)
		item = item.to(device)
		label = label.to(device)

		optimizer.zero_grad()
		prediction = model(user, item)
		loss = loss_function(prediction, label)
		loss.backward()
		optimizer.step()
		writer.add_scalar('MLP/loss/Train_loss', loss.item(), epoch)

	model.eval()
	HR, NDCG = metrics(model, test_loader, args.top_k, device)
	writer.add_scalar('MLP/Perfomance/HR@10', HR, epoch)
	writer.add_scalar('MLP/Perfomance/NDCG@10', NDCG, epoch)

	elapsed_time = time.time() - start_time
	print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
			time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
	print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

	if HR > best_hr:
		best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
		if args.out:
			if not os.path.exists(MODEL_PATH):
				os.mkdir(MODEL_PATH)
			torch.save(model, 
				'{}{}.pth'.format(MODEL_PATH, MODEL_MLP))

writer.close()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


The time elapse of epoch 001 is: 00: 00: 26
HR: 0.392	NDCG: 0.216
The time elapse of epoch 002 is: 00: 00: 26
HR: 0.393	NDCG: 0.222
The time elapse of epoch 003 is: 00: 00: 26
HR: 0.386	NDCG: 0.217
The time elapse of epoch 004 is: 00: 00: 25
HR: 0.400	NDCG: 0.219
The time elapse of epoch 005 is: 00: 00: 25
HR: 0.404	NDCG: 0.218
The time elapse of epoch 006 is: 00: 00: 25
HR: 0.422	NDCG: 0.231
The time elapse of epoch 007 is: 00: 00: 25
HR: 0.418	NDCG: 0.233
The time elapse of epoch 008 is: 00: 00: 25
HR: 0.422	NDCG: 0.239
The time elapse of epoch 009 is: 00: 00: 25
HR: 0.442	NDCG: 0.249
The time elapse of epoch 010 is: 00: 00: 26
HR: 0.460	NDCG: 0.259
The time elapse of epoch 011 is: 00: 00: 25
HR: 0.467	NDCG: 0.265
The time elapse of epoch 012 is: 00: 00: 26
HR: 0.473	NDCG: 0.274
The time elapse of epoch 013 is: 00: 00: 26
HR: 0.495	NDCG: 0.282
The time elapse of epoch 014 is: 00: 00: 26
HR: 0.508	NDCG: 0.288
The time elapse of epoch 015 is: 00: 00: 25
HR: 0.512	NDCG: 0.288
The time e

In [17]:
print("MLP best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(best_epoch, best_hr, best_ndcg))

MLP best epoch 020: HR = 0.546, NDCG = 0.302


## Training GMF model

In [18]:
writer = SummaryWriter()
# set model and loss, optimizer
model = Generalized_Matrix_Factorization(args, num_users, num_items)
model = model.to(device)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation
best_hr = 0
for epoch in range(1, args.gmf_epochs+1):
	model.train()
	start_time = time.time()

	for user, item, label in train_loader:
		user = user.to(device)
		item = item.to(device)
		label = label.to(device)

		optimizer.zero_grad()
		prediction = model(user, item)
		loss = loss_function(prediction, label)
		loss.backward()
		optimizer.step()
		writer.add_scalar('GMF/loss/Train_loss', loss.item(), epoch)

	model.eval()
	HR, NDCG = metrics(model, test_loader, args.top_k, device)
	writer.add_scalar('GMF/Perfomance/HR@10', HR, epoch)
	writer.add_scalar('GMF/Perfomance/NDCG@10', NDCG, epoch)

	elapsed_time = time.time() - start_time
	print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
			time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
	print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

	if HR > best_hr:
		best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
		if args.out:
			if not os.path.exists(MODEL_PATH):
				os.mkdir(MODEL_PATH)
			torch.save(model, 
				'{}{}.pth'.format(MODEL_PATH, MODEL_GMF))

writer.close()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


The time elapse of epoch 001 is: 00: 00: 22
HR: 0.092	NDCG: 0.043
The time elapse of epoch 002 is: 00: 00: 22
HR: 0.090	NDCG: 0.042
The time elapse of epoch 003 is: 00: 00: 21
HR: 0.104	NDCG: 0.047
The time elapse of epoch 004 is: 00: 00: 22
HR: 0.136	NDCG: 0.063
The time elapse of epoch 005 is: 00: 00: 22
HR: 0.190	NDCG: 0.091
The time elapse of epoch 006 is: 00: 00: 22
HR: 0.256	NDCG: 0.124
The time elapse of epoch 007 is: 00: 00: 22
HR: 0.310	NDCG: 0.155
The time elapse of epoch 008 is: 00: 00: 22
HR: 0.326	NDCG: 0.170
The time elapse of epoch 009 is: 00: 00: 22
HR: 0.349	NDCG: 0.188
The time elapse of epoch 010 is: 00: 00: 22
HR: 0.380	NDCG: 0.205
The time elapse of epoch 011 is: 00: 00: 22
HR: 0.403	NDCG: 0.216
The time elapse of epoch 012 is: 00: 00: 22
HR: 0.437	NDCG: 0.231
The time elapse of epoch 013 is: 00: 00: 21
HR: 0.453	NDCG: 0.241
The time elapse of epoch 014 is: 00: 00: 22
HR: 0.481	NDCG: 0.256
The time elapse of epoch 015 is: 00: 00: 22
HR: 0.495	NDCG: 0.263
The time e

In [19]:
print("GMF best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(best_epoch, best_hr, best_ndcg))

GMF best epoch 020: HR = 0.538, NDCG = 0.287


## Pushing SummaryWriter data to tensorboard

In [20]:
# !tensorboard dev upload --logdir /content/runs/ --name "Neural Collaborative filtering final" --description "Comparision of collaborative filtering of models" --one_shot


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/7d8H8rLKTzKBR92X5wnwDA/

[2021-11-25T12:35:02] Started scanning logdir.
[2021-11-25T12:35:35] Total uploaded: 96850 scalars, 0 tensors, 0 binary objects
[2021-11-25T12:35:35] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/7d8H8rLKTzKBR92X5wnwDA/


## Downloading model files and SummaryWriter data

In [21]:
'''
!zip -r /content/file.zip /content/models /content/runs
from google.colab import files
files.download('/content/file.zip') 
'''

  adding: content/models/ (stored 0%)
  adding: content/models/ml-100k_NeuMF.pth (deflated 8%)
  adding: content/models/ml-100k_GMF.pth (deflated 8%)
  adding: content/models/ml-100k_MLP.pth (deflated 8%)
  adding: content/runs/ (stored 0%)
  adding: content/runs/Nov25_12-13-06_a33fc4fe255d/ (stored 0%)
  adding: content/runs/Nov25_12-13-06_a33fc4fe255d/events.out.tfevents.1637842386.a33fc4fe255d (deflated 77%)
  adding: content/runs/Nov25_12-18-03_a33fc4fe255d/ (stored 0%)
  adding: content/runs/Nov25_12-18-03_a33fc4fe255d/events.out.tfevents.1637842683.a33fc4fe255d (deflated 76%)
  adding: content/runs/Nov25_12-26-57_a33fc4fe255d/ (stored 0%)
  adding: content/runs/Nov25_12-26-57_a33fc4fe255d/events.out.tfevents.1637843217.a33fc4fe255d (deflated 76%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>